In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
import whisper

model = whisper.load_model("medium")
# result = model.transcribe("audio.wav")

In [ ]:
from pathlib import Path
import hashlib
import subprocess
import json
from bs4 import BeautifulSoup


text_cache = Path('cache')

def sha1(input_string):
    """Helper to hash input strings"""
    try:

        # Step 5: Create a new SHA-1 hash object
        hash_object = hashlib.sha1()

        # Step 6: Update the hash object with the bytes-like object
        hash_object.update(input_string.encode('utf-8'))

        # Step 7: Get the hexadecimal representation of the hash
        return hash_object.hexdigest()
    except Exception as e:
        raise ValueError(input_string) from e


from functools import wraps
import inspect
import json
import pandas as pd
import hashlib


class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if hasattr(obj, 'to_dict'):
            return obj.to_dict()
        if isinstance(obj, pd.Int64Dtype):
            return int(obj)  # Convert Int64 to a regular int
        return json.JSONEncoder.default(self, obj)


def stored(func):
    """
    implements nix-like durable memoisation of function results.

    Lazy way to avoid recomputing expensive calls. Expects results to be JSON-serializable
    """
    @wraps(func)
    def CACHE(*args, **kwargs):
        name = func.__name__
        meta = {}

        meta["name"] = name
        meta["func"] = inspect.getsource(func)
        meta["args"] = args
        meta["kwargs"] = kwargs

        js = json.dumps(meta, cls=CustomEncoder)  # Using CustomEncoder
        sha = hashlib.sha1(js.encode('utf-8'))

        digest = sha.hexdigest()

        path = text_cache / f"{digest}-{name}.json"

        if path.exists():
            with path.open('r') as r:
                cached = json.load(r)
            return cached["result"]
        result = func(*args, **kwargs)
        meta["result"] = result
        with path.open('w') as w:
            json.dump(meta, w, cls=CustomEncoder)  # Using CustomEncoder
        return result

    return CACHE

In [56]:
import whisper

model = whisper.load_model("medium")


@stored
def transcribe(path: str):
    return model.transcribe(path)

In [54]:
import tensorflow as tf
import whisper

# Enable GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Load Whisper model
model = whisper.load_model("medium")

# Load audio file
audio = whisper.load_audio(path)

# Perform transcription
result = model.transcribe(audio)


/Users/valuamba/projs/components_agent_sales/venv/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [ ]:
01:32

In [62]:
path = '/Users/valuamba/Movies/2024-06-06 14-30-11.mkv'

In [59]:
path

'/Users/valuamba/Movies/2024-06-06 14-07-06.mkv'

In [63]:
result = transcribe(path)

/Users/valuamba/projs/components_agent_sales/venv/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [65]:
result

{'text': ' you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you',
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 2.06,
   'text': ' you',
   'tokens': [50364, 291, 50467],
   'temperature': 0.0,
   'avg_logprob': -0.44147413969039917,
   'compression_ratio': 0.2727272727272727,
   'no_speech_prob': 0.9342081546783447},
  {'id': 1,
   'seek': 3000,
   'start': 30.0,
   'end': 33.06,
   'text': ' you',
   'tokens': [50414, 291, 50517],
   'temperature': 0.0,
   'avg_logprob': -0.8645403385162354,
   'compression_ratio': 0.2727272727272727,
   'no_speech_prob': 0.7711876630783081},
  {'id': 2,
   'seek': 6000,
   'start': 60.0,
   'end': 63.06,
   'text': ' you',
   'tokens': [50414, 291, 50517],
   'temperature': 0.0,
   'avg_logprob': -0.6374342441558838,
   'compre

In [64]:
result['text']

' you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you'

In [ ]:
result = model.transcribe('/Users/valuamba/Movies/2024-06-04 14-42-39.mkv')

In [ ]:
result['text']

In [ ]:
with open('transcription.txt', 'w') as f:
    f.write(result['text'])

In [ ]:
import json

json_string = json.dumps(result, indent=4, ensure_ascii=False)
print(json_string)

with open('transcription.json', 'w') as f:
    f.write(json_string)

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
openai_client = OpenAI()

def get_gpt(content, model="gpt-4o", temperature=0, max_tokens=1000, stream=True):
    """
    Cached call to GPT.
    """
    messages = [{"role": "user", "content": content}]

    if stream:
        response = openai_client.chat.completions.create(
            model=model, 
            messages=messages, 
            temperature=temperature, 
            stream=True
        )
        
        collected_messages = []
        for chunk in response:
            if chunk.choices[0].delta.content:
                print(chunk.choices[0].delta.content, end='')
                collected_messages.append(chunk.choices[0].delta.content)
    
        content_str = ''.join(collected_messages)
        return content_str
    else:
        completion = openai_client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return completion.model_dump()


In [ ]:
text = """
По базе данных тоже все запросы. Количество запросов, которое прилетело в базу данных. Тут вопрос стоит по поводу инстанса. В инстанс или в саму базу данных. Начнем с того, что представляется возможным с инстанса. Мурбек. Я кому вещаю? А, я с закрытым микрофоном говорю. Да, да, да. С инстанса, с инстанса. Да, и количество запросов в байтах. Чтобы смотреть, видно должно быть ровненько в среднем. Ну так, плавненько. От дня в день. Если какие-то пошли спайки, то их надо будет смотреть, что это. Redis. Здесь стандартные метрики. Реквесты. Интересно. По всему. Весь входящий трафик в HRTech. Все, что в HRTech есть. Есть. Есть incoming трафик. И соответственно, процент ошибок. Все ошибки, которые у нас произошли. На процент ошибок посмотреть. 400-ки? Вообще, 400-ки, 500-ки, любые ошибки просто посмотреть. Все. Да, все ошибки. Там дальше будем разбираться. Может, будем уточнять вообще все ошибки за раз. И процент этих ошибок. И соответственно, трешхолд. И поэтому потом будем делать SLA и тому подобное. Так, processing time. Процессинг time. Процессинг time по реквесту. Скорее всего, мы можем получить где-нибудь из кубернетиса. Compute time мы, конечно, не сможем получить из кубернетиса. Нам надо будет это делать самим. Но это делаем в следующую атерацию. И не сейчас. CPU idle. Тут, понятно, надо разобраться, что мы отсюда можем получить. Здесь про CPU, про запас прочности, какой у нас имеется на текущий момент времени. Ну и вообще посмотреть в динамике, как это работает. Где у нас там high time, где у нас там low time. Примерно предсказать можно, но хотя бы понять в high time у нас сколько там осталось. 10% CPU или 80%. В среднем мы работаем на 10%. В high time до 20% поднимаемся. И 80% CPU у нас просто так стоит. Зачем оно нам тогда нужно? Дальше. Клиенты. Это наши клиентские библиотечки, через которые мы ходим в другие сервисы. То есть я нахожусь в сервисе A, хочу сделать запрос в сервис B. Для этого у меня есть клиентская библиотека для сервиса B. Я ее беру и начинаю работать с сервисом B через эту клиентскую библиотеку. В эти клиентские библиотеки необходимо внедрить VBHTTP нашу библиотечку, в которую добавят метрики все исходящие запросы из наших клиентских библиотек. И сколько там ошибок было, сколько тайм-аутов было и реквеста в персентилях. Так, отдельно вынесено RBC и AUS. RBC и AUS – это два самых нагрузочных сервисов. Их прямо отдельно. Высоконагруженных. Да, спасибо. Ну ладно, да. Давай-давай, еще какой-то? Да, просто аутхен перестает быть. Ну, пока так. Это так, да. RBC и аутхен, верно. Пока так, будем смотреть вообще. Если он перестанет быть таким, у него там в ноль все сойдет и... У него он сейчас, по сути, будет обрабатывать всем работу. Ну, пока что нужно. RBC точно будут все ходить. Так, топ-10 сервисов по RequestCant. Посмотреть вообще, кто у нас в топе. И Гошные метрики. Тут стандартные. На самом деле, по Гошным метрикам на моей предыдущей работе Гошные метрики – это называлось не Гошные метрики, а Performance. И там прямо отдельная, на самом деле, дашбарда. Там, конечно, все было и allocation, и горутин. Но там еще дополнительные какие-то формулы были. По Гошным метрикам... Ну, что мне было бы интересно? Наверное, горутины в основном. Горутины в основном. Есть они у нас. То есть они должны утилизироваться равномерно. Вот. Да-да-да, горутин у нас есть. А, и Temporal. Там по workflow, по activities. Сколько было запусков. Там вообще в Temporal прям фу-фу-фу, целая огромная морда.
"""

len(text)

In [ ]:
def chunk_text(text, chunk_size):
    # Loop over the text in increments of chunk_size
    for i in range(0, len(text), chunk_size):
        yield text[i:i + chunk_size]

# Example usage
long_text = "This is a very long text that we want to split into smaller chunks."
chunk_size = 4000



In [ ]:
@stored
def get_metrics_from_context(snipped: str):
    return get_gpt(f"""
Перечисли метрики, которые упоминались в тексте:

{snipped}""", max_tokens=4000)

In [ ]:
@stored
def test_stored(resut):
    return result

In [ ]:
test_stored(result)

In [ ]:
responces = []

for chunk in chunk_text(result['text'], chunk_size):
    responces.append(get_metrics_from_context(chunk))

In [53]:
get_gpt(f"""
Это лог моего разговора с руководитлем я Technical Project Manager, мы разрабатываем на Golang.
Про какие задачи мы тут говорили и о чем договорились?

{result['text']}
""")

В вашем разговоре с руководителем обсуждались следующие задачи и договоренности:

1. **Метрики по активным мобильным сессиям**:
   - Необходимо запросить у Анастасии метрики по количеству людей, у которых нет активной мобильной сессии.
   - Уточнить метрики УАУСа.

2. **Переезд на новые кластера**:
   - Все тикеты по АКСЕС и ЕЛ отправить на доску, где их будет разбирать девопсовая команда и стажеры.
   - Ожидать решения по бордам в течение недели.

3. **Интеграция поиска по сотруднику соц. информации**:
   - Виталий сделал ручку поиска, которая дает ОРКС структуру и информацию о пользователе.
   - Тестирование этой ручки и ревью, которое висит на Вансевиче. Возможно, попросить Влада помочь с ревью.

4. **Обработка данных БАСКов и ОРКС**:
   - Наставничество стажеров, которые будут помогать Ярику с переходом на новые рельсы и обратно в инфу.

5. **Докхаб и дата-сорсы**:
   - Распарсили все сервисы и ямлы сервисов HR-теха, собрали общий JSON и научились импортировать в докхаб.
   - Необх

'В вашем разговоре с руководителем обсуждались следующие задачи и договоренности:\n\n1. **Метрики по активным мобильным сессиям**:\n   - Необходимо запросить у Анастасии метрики по количеству людей, у которых нет активной мобильной сессии.\n   - Уточнить метрики УАУСа.\n\n2. **Переезд на новые кластера**:\n   - Все тикеты по АКСЕС и ЕЛ отправить на доску, где их будет разбирать девопсовая команда и стажеры.\n   - Ожидать решения по бордам в течение недели.\n\n3. **Интеграция поиска по сотруднику соц. информации**:\n   - Виталий сделал ручку поиска, которая дает ОРКС структуру и информацию о пользователе.\n   - Тестирование этой ручки и ревью, которое висит на Вансевиче. Возможно, попросить Влада помочь с ревью.\n\n4. **Обработка данных БАСКов и ОРКС**:\n   - Наставничество стажеров, которые будут помогать Ярику с переходом на новые рельсы и обратно в инфу.\n\n5. **Докхаб и дата-сорсы**:\n   - Распарсили все сервисы и ямлы сервисов HR-теха, собрали общий JSON и научились импортировать в

In [ ]:
get_gpt(f"""
Напиши UserStory, которая будет понятна DevOps разработчику по добавлению Grafana дашбордов, используй meeting logs:

{'\n'.join(responces)}
""", max_tokens=4000)

In [ ]:
len(result['text'])

In [ ]:
result['text']

In [ ]:
result

In [ ]:
get_gpt("how to map DataFrame object to into dataclass with comprehetion?")